### Simplified DJ for Single Spot Scotopic Dates

In [1]:
import datajoint as dj
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import glob

import sys
print('Importing Modules')
sys.path.append('../dj_modules/')
import dj_metadata as djm
import dj_populate as djp

import pandas as pd
print('Importing more modules')
sys.path.append('../data_modules/')
import spikeoutputs as so
import spikeplots as sp

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


[2024-11-04 13:26:57,022][INFO]: Connecting root@127.0.0.1:3306
[2024-11-04 13:26:57,113][INFO]: Connected root@127.0.0.1:3306


Importing Modules
Importing more modules


In [2]:
dj.config['database.host'] = '127.0.0.1'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'simple'
dj.conn()

DataJoint connection (connected) root@127.0.0.1:3306

In [3]:
print(djp.STR_NAS_PATH)
print(djp.STR_ANALYSIS_PATH)
print(djp.STR_SORT_PATH)

/Volumes/data/
/Volumes/data/analysis/
/Volumes/data/data/sorted/


In [7]:
str_metadata_dir = '/Volumes/data/data/metadata/json'

In [8]:
# This may take a few minutes if running for the first time.
djm.load_metadata(str_metadata_dir)
djp.load_chunks_and_files()
djp.load_typing()

Added 0 new experiments
20230825C not in Experiment table.
20230825C not in Experiment table.
20230825C not in Experiment table.
20240606H not in Experiment table.
20240611H not in Experiment table.
20240611H not in Experiment table.
20240627H not in Experiment table.
20240627H not in Experiment table.
20240717H not in Experiment table.
20240717H not in Experiment table.
20240717H not in Experiment table.
Added 0 new data files


In [4]:
djp.load_typing_notes(str_csv = '/Users/srinidhienaidu/Downloads/Copy of SortedData - Sheet1.csv')

Inserted 111 typing notes


In [5]:
ls_protocols = djm.search_protocol('Singlespot')
print(ls_protocols)

['edu.washington.riekelab.protocols.SingleSpot']


In [6]:
df_meta = djm.meta_from_protocol(ls_protocols)
df_ct = djm.celltyping_from_meta(df_meta)

In [7]:
df_ct

num_cells  \
date_id   protocol_id                     group_idx noise_data_file block_idx chunk_id noise_data_files algorithm b_typing_file_exists typing_file                                                     
20230111C manookinlab.protocols.FastNoise 0         data019         0         chunk1   data019          kilosort2 1                    /Volumes/data/analysis/20230111C/chunk1/kilosor...       1115   
                                                                                                                                       /Volumes/data/analysis/20230111C/chunk1/kilosor...       1115   
20230228C manookinlab.protocols.FastNoise 0         data045         2         chunk3   data045          kilosort2 1                    /Volumes/data/analysis/20230228C/chunk3/kilosor...        991   
20230322C manookinlab.protocols.FastNoise 2         data005         0         chunk1   data005_data006  kilosort2 1                    /Volumes/data/analysis/20230322C/chunk1/kilosor...        879   
                                                                                                                                       /Volumes/data/analysis/20230322C/chunk1/kilosor...        879   
20230328C manookinlab.protocols.FastNoise 1         data005         0         chunk1   data005_data006  kilosort2 1                    /Volumes/data/analysis/20230328C/chunk1/kilosor...        323   
20230524C manookinlab.protocols.FastNoise 2         data003         0         chunk2   data003          kilosort2 1                    /Volumes/data/analysis/20230524C/chunk2/kilosor...        466   
                                                                                                                                       /Volumes/data/analysis/20230524C/chunk2/kilosor...        466   
20230713C manookinlab.protocols.FastNoise 2         data027         0         chunk2   data027_data024  kilosort2 1                    /Volumes/data/analysis/20230713C/chunk2/kilosor...        581   
20230725C manookinlab.protocols.FastNoise 0         data018         1         chunk3   data018_data019  kilosort2 1                    /Volumes/data/analysis/20230725C/chunk3/kilosor...        516   

                                                                                                                                                                                           num_goodcells  \
date_id   protocol_id                     group_idx noise_data_file block_idx chunk_id noise_data_files algorithm b_typing_file_exists typing_file                                                         
20230111C manookinlab.protocols.FastNoise 0         data019         0         chunk1   data019          kilosort2 1                    /Volumes/data/analysis/20230111C/chunk1/kilosor...           1115   
                                                                                                                                       /Volumes/data/analysis/20230111C/chunk1/kilosor...           1115   
20230228C manookinlab.protocols.FastNoise 0         data045         2         chunk3   data045          kilosort2 1                    /Volumes/data/analysis/20230228C/chunk3/kilosor...            991   
20230322C manookinlab.protocols.FastNoise 2         data005         0         chunk1   data005_data006  kilosort2 1                    /Volumes/data/analysis/20230322C/chunk1/kilosor...            879   
                                                                                                                                       /Volumes/data/analysis/20230322C/chunk1/kilosor...            879   
20230328C manookinlab.protocols.FastNoise 1         data005         0         chunk1   data005_data006  kilosort2 1                    /Volumes/data/analysis/20230328C/chunk1/kilosor...            323   
20230524C manookinlab.protocols.FastNoise 2         data003         0         chunk2   data003          kilosort2 1                    /Volumes/data/analy

In [ ]:
df_meta = df_meta.reset_index()
filtered_df = df_meta[df_meta["group_label"] == 'scotopic flashes'].reset_index()
filtered_df['date_chunk'] = filtered_df['date_id'] + '_' + filtered_df['chunk_id']
filtered_df['quality'] = ''

In [12]:
df_notes = djm.TypingNotes().fetch(format='frame').reset_index()

df_notes['date_chunk'] = df_notes['date_id'] + '_' + df_notes['chunk_id']
for idx in filtered_df.index:
    str_datechunk = filtered_df.loc[idx, 'date_chunk']
    filtered_df.loc[idx, 'quality'] = df_notes[df_notes['date_chunk']==str_datechunk]['quality'].values[0]

In [15]:
filtered_df

,index,date_id,protocol_id,group_idx,data_file,block_idx,chunk_id,frame_times,n_epochs,group_label,source_label,date_chunk,quality
0,26,20230322C,edu.washington.riekelab.protocols.SingleSpot,0,data000,0,chunk1,"[[1, 6, 26, 39, 59, 72, 92, 105, 125, 138, 158...",200,scotopic flashes,,20230322C_chunk1,High
1,27,20230322C,edu.washington.riekelab.protocols.SingleSpot,0,data001,1,chunk1,"[[1, 6, 26, 39, 59, 72, 92, 105, 125, 138, 158...",200,scotopic flashes,,20230322C_chunk1,High
2,28,20230322C,edu.washington.riekelab.protocols.SingleSpot,0,data002,2,chunk1,"[[1, 6, 26, 39, 59, 72, 92, 105, 125, 138, 158...",200,scotopic flashes,,20230322C_chunk1,High
3,29,20230322C,edu.washington.riekelab.protocols.SingleSpot,0,data003,3,chunk1,"[[1, 6, 26, 39, 59, 72, 92, 105, 125, 138, 158...",200,scotopic flashes,,20230322C_chunk1,High
4,30,20230322C,edu.washington.riekelab.protocols.SingleSpot,0,data004,4,chunk1,"[[1, 6, 26, 39, 59, 72, 92, 105, 125, 138, 158...",200,scotopic flashes,,20230322C_chunk1,High
5,31,20230328C,edu.washington.riekelab.protocols.SingleSpot,0,data000,0,chunk1,"[[1, 6, 26, 39, 59, 72, 92, 105, 125, 138, 158...",200,scotopic flashes,,20230328C_chunk1,Low
6,32,20230328C,edu.washington.riekelab.protocols.SingleSpot,0,data001,1,chunk1,"[[1, 6, 26, 39, 59, 72, 92, 105, 125, 138, 159...",200,scotopic flashes,,20230328C_chunk1,Low
7,33,20230328C,edu.washington.riekelab.protocols.SingleSpot,0,data002,2,chunk1,"[[1, 6, 26, 39, 59, 72, 92, 105, 125, 138, 159...",200,scotopic flashes,,20230328C_chunk1,Low
8,34,20230328C,edu.washington.riekelab.protocols.SingleSpot,0,data003,3,chunk1,"[[1, 6, 26, 39, 59, 72, 92, 105, 125, 138, 159...",200,scotopic flashes,,20230328C_chunk1,Low
9,35,20230328C,edu.washington.riekelab.protocols.SingleSpot,0,data004,4,chunk1,"[[1, 6, 26, 39, 59, 72, 92, 105, 125, 138, 159...",200,scotopic flashes,,20230328C_chunk1,Low


In [1]:
# for idx in range(len(dates)):
#     str_date = dates[idx]
#     str_chunk = chunks[idx]
#     type_info = (djm.TypingNotes() & f'date_id="{str_date}"' & f'chunk_id="{str_chunk}"' & f'quality="{quality_desired}"').fetch(format = 'frame')

#     print(dates[idx], chunks[idx])